Tutorial followed: https://www.youtube.com/watch?v=kSqxn6zGE0c

### Prerequisites
1. Check Python installation version (idk which one works, i use 3.11 during coding :p)

2. Install required packages via terminal: ```pip install -r requirements.txt```

In [4]:
# Image Preparation Imports
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [5]:
# Reading in images
glob()


TypeError: glob() missing 1 required positional argument: 'pathname'

In [3]:

img_mpl = plt.imread()

img_cv2 = cv2.imread()




TypeError: imread() missing 1 required positional argument: 'fname'

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.axis('off')
ax.imshow(img_mpl)
